<a href="https://colab.research.google.com/github/haokeliu/colaboratory/blob/master/DeepLearn_Notes/%E6%95%B0%E6%8D%AE%E5%92%8C%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Load pytorch library
!pip3 install torch

In [0]:
from argparse import Namespace
import collections
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random 
import torch

In [0]:
# Set Numpy and PyTorch seeds
def set_seeds(seed,cuda):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if cuda:
    torch.cuda.manual_seed_all(seed)

In [8]:
#Arguments
args = Namespace(
    seed = 1234,
    cuda = False,
    shuffle = True,
    data_file = 'tumors.csv',
    reduced_data_file = 'tumors_reduced.csv',
    train_size = 0.75,
    test_size = 0.25,
    num_hidden_units = 100,
    learning_rate = 1e-3,
    num_epochs = 100,
)
set_seeds(seed = args.seed, cuda = args.cuda)
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

Using CUDA: False
